In [ ]:
from pathlib import Path
import torch
from torch import amp
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
import numpy as np
import onnx
from tqdm import tqdm

In [ ]:
# ---------------- CONFIG ----------------
DATASET_DIR = Path("id_dataset")
IMG_SIZE = 224
NUM_CLASSES = 2
EPOCHS = 15
FREEZE_EPOCHS = 5
BATCH_SIZE = 8
LR = 1e-4
WEIGHT_DECAY = 1e-2
VALID_SPLIT = 0.2
SEED = 42

DEVICE = (
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

torch.manual_seed(SEED)

In [ ]:
# ---------------- DATA AUGMENTATION ----------------
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.6, 1.0)),
    transforms.RandomHorizontalFlip(p=0.8),
    transforms.RandomRotation(25),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.2),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.05, 0.25)),
])

val_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])

# ---------------- DATASETS ----------------
full_dataset = datasets.ImageFolder(DATASET_DIR, transform=train_tf)
num_val = int(len(full_dataset) * VALID_SPLIT)
num_train = len(full_dataset) - num_val
train_dataset, val_dataset = random_split(full_dataset, [num_train, num_val])
val_dataset.dataset.transform = val_tf

# ---- Class balancing ----
labels = [full_dataset.targets[i] for i in train_dataset.indices]
class_counts = np.bincount(labels)
weights = 1.0 / class_counts
sample_weights = [weights[y] for y in labels]

sampler = WeightedRandomSampler(
    sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          sampler=sampler, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)

print("Classes:", full_dataset.classes)

# ---------------- MODEL ----------------
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# 🔒 SINGLE-LAYER FC — EXPORT SAFE
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

model.to(DEVICE)

# ---- Freeze backbone ----
for name, param in model.named_parameters():
    if not name.startswith("fc"):
        param.requires_grad = False

# ---------------- OPTIMIZATION ----------------
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS
)

scaler = amp.GradScaler()

# ---------------- TRAINING ----------------
best_val_acc = 0.0
patience = 5
no_improve = 0

for epoch in range(EPOCHS):

    if epoch == FREEZE_EPOCHS:
        print("🔓 Unfreezing backbone")
        for param in model.parameters():
            param.requires_grad = True
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=LR,
            weight_decay=WEIGHT_DECAY
        )

    # ---- TRAIN ----
    model.train()
    train_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1} [TRAIN]"):
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        with amp.autocast(device_type=DEVICE):
            outputs = model(imgs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total

    # ---- VALIDATION ----
    model.eval()
    val_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            with amp.autocast(device_type=DEVICE):
                outputs = model(imgs)
                loss = criterion(outputs, labels)

            val_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    scheduler.step()

    print(f"""
Epoch {epoch+1}/{EPOCHS}
Train Acc: {train_acc:.2f}%
Val   Acc: {val_acc:.2f}%
LR: {scheduler.get_last_lr()[0]:.2e}
""")

    # ---- Checkpoint ----
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "cats_resnet50_best.pth")
        print("✔ Saved best model")
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= patience:
            print("⚠ Early stopping")
            break

print("✅ Training complete")


Classes: ['devi', 'sati']


/var/folders/jc/0m_v96ps7h12t3pj03psgnwr0000gn/T/ipykernel_8253/4055074135.py:103: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  scaler = amp.GradScaler()
Epoch 1 [TRAIN]: 100%|██████████| 173/173 [00:20<00:00,  8.24it/s]



Epoch 1/15
Train Acc: 79.44%
Val   Acc: 91.01%
LR: 9.89e-05

✔ Saved best model


Epoch 2 [TRAIN]: 100%|██████████| 173/173 [00:19<00:00,  8.72it/s]



Epoch 2/15
Train Acc: 89.07%
Val   Acc: 93.33%
LR: 9.57e-05

✔ Saved best model


...


Epoch 10/15
Train Acc: 99.78%
Val   Acc: 99.71%
LR: 2.50e-05



Epoch 11 [TRAIN]: 100%|██████████| 173/173 [00:44<00:00,  3.87it/s]



Epoch 11/15
Train Acc: 100.00%
Val   Acc: 100.00%
LR: 1.65e-05



Epoch 12 [TRAIN]: 100%|██████████| 173/173 [00:44<00:00,  3.87it/s]



Epoch 12/15
Train Acc: 99.86%
Val   Acc: 100.00%
LR: 9.55e-06

⚠ Early stopping
✅ Training complete


In [ ]:

def export_resnet50_onnx(weights_path: str, output_path: str):
    """
    Export ResNet50 to ONNX compatible with OpenCV DNN on Raspberry Pi
    Assumes training used: model.fc = nn.Linear(...)
    """
    NUM_CLASSES = 2
    IMG_SIZE = 224

    print("🔄 Building ResNet50 architecture (training-compatible)...")

    # ---- EXACT SAME ARCHITECTURE AS TRAINING ----
    model = models.resnet50(weights=None)
    model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

    print("🔄 Loading trained weights...")
    state = torch.load(weights_path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval()

    print("🔄 Creating dummy input...")
    dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE)

    print("🔄 Exporting to ONNX (OpenCV-safe)...")
    torch.onnx.export(
        model,
        dummy_input,
        output_path,
        input_names=["image"],
        output_names=["logits"],   # raw logits (1, 2)
        opset_version=11,          # safest for OpenCV DNN
        do_constant_folding=True,
        dynamic_axes=None          # STATIC SHAPES (critical)
    )

    print(f"✅ Exported ONNX: {output_path}")

    # ---- Validate ----
    onnx_model = onnx.load(output_path)
    onnx.checker.check_model(onnx_model)
    print("✅ ONNX model is valid")

    # ---- Repack into single file ----
    single_file = output_path.replace(".onnx", "_single.onnx")
    onnx.save_model(
        onnx_model,
        single_file,
        save_as_external_data=False,
        all_tensors_to_one_file=True
    )

    size_mb = Path(single_file).stat().st_size / (1024 * 1024)
    print(f"✔ Single-file ONNX: {single_file} ({size_mb:.1f} MB)")


In [13]:
export_resnet50_onnx(
    weights_path="cats_resnet50_best.pth",
    output_path="cats_resnet50.onnx"
)

🔄 Building ResNet50 architecture (training-compatible)...
🔄 Loading trained weights...
🔄 Creating dummy input...
🔄 Exporting to ONNX (OpenCV-safe)...


W0210 14:10:15.090276 8253 site-packages/torch/onnx/_internal/exporter/_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0210 14:10:15.683768 8253 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0210 14:10:15.684347 8253 site-packages/torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale

[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/copyreg.py:101: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 11).
Failed to convert the model to the target version 11 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
  File "/Library/Frameworks/Python.framework/Versions

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 106 of general pattern rewrite rules.
✅ Exported ONNX: cats_resnet50.onnx
✅ ONNX model is valid
✔ Single-file ONNX: cats_resnet50_single.onnx (89.8 MB)
